In [1]:
import mysql.connector
import boto3
from boto3.dynamodb.conditions import Key, Attr
from datetime import datetime
from decimal import Decimal
import csv

In [2]:
#colocar aqui os dados necessarios para o filtro
# data1 = "2020-10-01"
# data2 = "2020-12-31"
data1 = "2021-10-01"
data2 = "2021-12-31"
cliente = "(92218)"

In [3]:
#conexão com o banco de dados para capturar as solicitações desejadas
con = mysql.connector.connect(host="#######", 
                              user="#######", passwd="##############", db="#######")
cursor = con.cursor(buffered=True)
#todos os clientes
# query = "select distinct(s.id) from solicitacao s inner join pretendente p on p.solicitacao_id = s.id inner join pretendente_produto pp on pp.pretendente_id = p.id where pp.payment_at between '"+data1+"' and '"+data2+"'"
#filtro de clientes
query = "select distinct(s.id) from solicitacao s inner join pretendente p on p.solicitacao_id = s.id inner join pretendente_produto pp on pp.pretendente_id = p.id where pp.payment_at between '"+data1+"' and '"+data2+"' and s.cliente_id in "+cliente+""
cursor.execute(query)
solicitacoes = []
for resultado in cursor:
    solicitacoes.append(resultado[0])

In [4]:
#Fazendo conexao com o banco aws para pegar somente as solicitacoes retornadas no banco mysql
dynamodb = boto3.resource('dynamodb',
                          region_name='us-east-1',
                          aws_access_key_id='#######',
                          aws_secret_access_key='######')
table = dynamodb.Table('evaluation-report-data-prod')
responses = []
for i in range(len(solicitacoes)):
    responses.append(table.query(KeyConditionExpression=Key('solicitation_id').eq(solicitacoes[i])))
    print('pegando informações da solicitação nº ', solicitacoes[i])

pegando informações da solicitação nº  50985
pegando informações da solicitação nº  51082
pegando informações da solicitação nº  51088
pegando informações da solicitação nº  51147
pegando informações da solicitação nº  51172
pegando informações da solicitação nº  51346
pegando informações da solicitação nº  51436
pegando informações da solicitação nº  51499
pegando informações da solicitação nº  51502
pegando informações da solicitação nº  51504
pegando informações da solicitação nº  50840
pegando informações da solicitação nº  51538
pegando informações da solicitação nº  51616
pegando informações da solicitação nº  51646
pegando informações da solicitação nº  51674
pegando informações da solicitação nº  51711
pegando informações da solicitação nº  51763
pegando informações da solicitação nº  51872
pegando informações da solicitação nº  51962
pegando informações da solicitação nº  51997
pegando informações da solicitação nº  52001
pegando informações da solicitação nº  52058
pegando in

pegando informações da solicitação nº  56950
pegando informações da solicitação nº  57012
pegando informações da solicitação nº  57040
pegando informações da solicitação nº  57147
pegando informações da solicitação nº  57149
pegando informações da solicitação nº  57159
pegando informações da solicitação nº  57250
pegando informações da solicitação nº  57293
pegando informações da solicitação nº  57304
pegando informações da solicitação nº  57307
pegando informações da solicitação nº  57320
pegando informações da solicitação nº  57339
pegando informações da solicitação nº  57340
pegando informações da solicitação nº  57344
pegando informações da solicitação nº  57375
pegando informações da solicitação nº  57405
pegando informações da solicitação nº  57406
pegando informações da solicitação nº  57442
pegando informações da solicitação nº  57478
pegando informações da solicitação nº  57491
pegando informações da solicitação nº  57493
pegando informações da solicitação nº  57575
pegando in

In [5]:
relatorio = {}
relatorio['locacao_geral'] = {}
relatorio['locacao_fiadores'] = {}
relatorio['locacao_inquilinos'] = {}
relatorio['individual_fiadores'] = {}
relatorio['individual_inquilinos'] = {}
relatorio['individual_fiadores_estudante'] = {}
relatorio['individual_outros'] = {}     

In [6]:
def atualiza_parecer(parecer):
    if parecer == 'MUITO BOM':
        return 'RISCO MUITO BAIXO'
    elif parecer == 'BOM':
        return 'RISCO BAIXO'
    elif parecer == 'REGULAR':
        return 'RISCO MÉDIO'
    elif parecer == 'INCONSISTENTE':
        return 'RISCO ACIMA DA MÉDIA'
    elif parecer == 'DESACONSELHÁVEL':
        return 'RISCO MUITO ALTO'

for i in range(len(responses)):
    print('Solicitação: ', solicitacoes[i])
    laudo_locacao_inquilinos = ''
    laudo_locacao_fiadores = ''
    for j in range(len(responses[i]['Items'])):
        if(responses[i]['Items'][j]['applicant_id'] == 0):
            if 'parecer_locacao' in responses[i]['Items'][j]['data']:
                if 'parecer' in responses[i]['Items'][j]['data']['parecer_locacao']['parecer_fiadores']:
                    parecer_fiadores = responses[i]['Items'][j]['data']['parecer_locacao']['parecer_fiadores']['parecer']
                    print('Parecer da locação (fiadores): ', parecer_fiadores)
                    if parecer_fiadores in ['MUITO BOM', 'BOM', 'REGULAR', 'INCONSISTENTE', 'DESACONSELHÁVEL']:
                        parecer_fiadores = atualiza_parecer(parecer_fiadores)
                        print('>> ATUALIZADO PARA: ', parecer_fiadores)
                    if parecer_fiadores not in relatorio['locacao_fiadores']:
                        relatorio['locacao_fiadores'][parecer_fiadores] = 0
                    relatorio['locacao_fiadores'][parecer_fiadores] = relatorio['locacao_fiadores'][parecer_fiadores] + 1
                    
                if 'parecer' in responses[i]['Items'][j]['data']['parecer_locacao']['parecer_inquilinos']:
                    parecer_inquilinos = responses[i]['Items'][j]['data']['parecer_locacao']['parecer_inquilinos']['parecer']
                    print('Parecer da locação (inquilinos): ', parecer_inquilinos)
                    if parecer_inquilinos in ['MUITO BOM', 'BOM', 'REGULAR', 'INCONSISTENTE', 'DESACONSELHÁVEL']:
                        parecer_inquilinos = atualiza_parecer(parecer_inquilinos)
                        print('>> ATUALIZADO PARA: ', parecer_inquilinos)
                    if parecer_inquilinos not in relatorio['locacao_inquilinos']:
                        relatorio['locacao_inquilinos'][parecer_inquilinos] = 0
                    relatorio['locacao_inquilinos'][parecer_inquilinos] = relatorio['locacao_inquilinos'][parecer_inquilinos] + 1    
        else:
            parecer = responses[i]['Items'][j]['data']['laudo']['parecer_sistemico'][0]['parecer']
            print('Parecer individual: ', parecer)
            if parecer in ['MUITO BOM', 'BOM', 'REGULAR', 'INCONSISTENTE', 'DESACONSELHÁVEL']:
                parecer = atualiza_parecer(parecer)
                print('>> ATUALIZADO PARA: ', parecer)
            if responses[i]['Items'][j]['data']['pretendente']['tipo_pretendente'] in ['INQUILINO', 'FIADOR', 'FIADOR DE ESTUDANTE']:
                if responses[i]['Items'][j]['data']['pretendente']['tipo_pretendente'] == 'INQUILINO':
                    if parecer not in relatorio['individual_inquilinos']:
                        relatorio['individual_inquilinos'][parecer] = 0
                    relatorio['individual_inquilinos'][parecer] = relatorio['individual_inquilinos'][parecer] + 1
                elif responses[i]['Items'][j]['data']['pretendente']['tipo_pretendente'] == 'FIADOR':
                    if parecer not in relatorio['individual_fiadores']:
                        relatorio['individual_fiadores'][parecer] = 0
                    relatorio['individual_fiadores'][parecer] = relatorio['individual_fiadores'][parecer] + 1
                elif responses[i]['Items'][j]['data']['pretendente']['tipo_pretendente'] == 'FIADOR DE ESTUDANTE':
                    if parecer not in relatorio['individual_fiadores_estudante']:
                        relatorio['individual_fiadores_estudante'][parecer] = 0
                    relatorio['individual_fiadores_estudante'][parecer] = relatorio['individual_fiadores_estudante'][parecer] + 1
            else:
                if parecer not in relatorio['individual_outros']:
                    relatorio['individual_outros'][parecer] = 0
                relatorio['individual_outros'][parecer] = relatorio['individual_outros'][parecer] + 1
            
    print('-----------')

Solicitação:  50985
Parecer da locação (fiadores):  RISCO MUITO BAIXO
Parecer da locação (inquilinos):  RISCO MUITO BAIXO
Parecer individual:  RISCO MUITO BAIXO
Parecer individual:  RISCO MUITO BAIXO
Parecer individual:  NÃO APURADO
-----------
Solicitação:  51082
Parecer da locação (inquilinos):  RISCO MUITO BAIXO
Parecer individual:  RISCO MUITO BAIXO
-----------
Solicitação:  51088
Parecer da locação (inquilinos):  RISCO ACIMA DA MÉDIA
Parecer individual:  RISCO ACIMA DA MÉDIA
Parecer individual:  NÃO APURADO
-----------
Solicitação:  51147
Parecer da locação (fiadores):  RISCO MÉDIO
Parecer individual:  NÃO APURADO
Parecer individual:  RISCO ACIMA DA MÉDIA
Parecer individual:  NÃO APURADO
Parecer individual:  RISCO ACIMA DA MÉDIA
Parecer individual:  RISCO ACIMA DA MÉDIA
-----------
Solicitação:  51172
Parecer da locação (fiadores):  RISCO BAIXO
Parecer da locação (inquilinos):  RISCO MUITO BAIXO
Parecer individual:  RISCO MUITO BAIXO
Parecer individual:  RISCO BAIXO
-----------
So

Parecer individual:  RISCO MUITO BAIXO
-----------
Solicitação:  53921
Parecer da locação (inquilinos):  RISCO ACIMA DA MÉDIA
Parecer individual:  RISCO ACIMA DA MÉDIA
-----------
Solicitação:  53922
Parecer individual:  NÃO APURADO
-----------
Solicitação:  53924
Parecer da locação (inquilinos):  RISCO MÉDIO
Parecer individual:  RISCO MÉDIO
-----------
Solicitação:  53932
Parecer da locação (inquilinos):  NÃO APURADO
Parecer individual:  NÃO APURADO
-----------
Solicitação:  53962
Parecer da locação (inquilinos):  RISCO MÉDIO
Parecer individual:  RISCO MÉDIO
Parecer individual:  CPF | Pendente de Regularização
-----------
Solicitação:  53977
Parecer da locação (inquilinos):  RISCO MÉDIO
Parecer individual:  RISCO MÉDIO
Parecer individual:  RISCO MÉDIO
-----------
Solicitação:  54004
Parecer da locação (inquilinos):  RISCO MÉDIO
Parecer individual:  RISCO MÉDIO
-----------
Solicitação:  54006
Parecer da locação (inquilinos):  RISCO BAIXO
Parecer individual:  RISCO BAIXO
-----------
Sol

Parecer individual:  RISCO ACIMA DA MÉDIA
Parecer individual:  RISCO ACIMA DA MÉDIA
-----------
Solicitação:  58811
Parecer da locação (inquilinos):  RISCO MÉDIO
Parecer individual:  RISCO MÉDIO
-----------
Solicitação:  58869
Parecer da locação (fiadores):  RISCO MUITO BAIXO
Parecer da locação (inquilinos):  RISCO BAIXO
Parecer individual:  RISCO BAIXO
Parecer individual:  RISCO ACIMA DA MÉDIA
Parecer individual:  RISCO ACIMA DA MÉDIA
-----------
Solicitação:  58926
Parecer da locação (inquilinos):  RISCO MUITO BAIXO
Parecer individual:  RISCO MUITO BAIXO
-----------
Solicitação:  58929
Parecer da locação (fiadores):  RISCO BAIXO
Parecer individual:  RISCO BAIXO
-----------
Solicitação:  58967
Parecer da locação (fiadores):  RISCO ACIMA DA MÉDIA
Parecer da locação (inquilinos):  RISCO ACIMA DA MÉDIA
Parecer individual:  RISCO MUITO ALTO
Parecer individual:  RISCO ACIMA DA MÉDIA
Parecer individual:  RISCO ACIMA DA MÉDIA
-----------
Solicitação:  59046
Parecer da locação (inquilinos):  

In [7]:
relatorio

{'locacao_geral': {},
 'locacao_fiadores': {'RISCO MUITO BAIXO': 48,
  'RISCO MÉDIO': 15,
  'RISCO BAIXO': 28,
  'RISCO ACIMA DA MÉDIA': 21,
  'RISCO MUITO ALTO': 4,
  'NÃO APURADO': 1},
 'locacao_inquilinos': {'RISCO MUITO BAIXO': 40,
  'RISCO ACIMA DA MÉDIA': 66,
  'RISCO MÉDIO': 106,
  'RISCO MUITO ALTO': 22,
  'RISCO BAIXO': 51,
  'NÃO APURADO': 4},
 'individual_fiadores': {'RISCO MUITO BAIXO': 43,
  'RISCO ACIMA DA MÉDIA': 40,
  'NÃO APURADO': 7,
  'RISCO BAIXO': 33,
  'RISCO MÉDIO': 21,
  'RISCO MUITO ALTO': 9},
 'individual_inquilinos': {'RISCO MUITO BAIXO': 41,
  'RISCO ACIMA DA MÉDIA': 110,
  'RISCO MÉDIO': 101,
  'RISCO MUITO ALTO': 43,
  'NÃO APURADO': 3,
  'RISCO BAIXO': 45,
  'CPF | Pendente de Regularização': 6},
 'individual_fiadores_estudante': {},
 'individual_outros': {'NÃO APURADO': 40,
  'RISCO ACIMA DA MÉDIA': 17,
  'RISCO MUITO BAIXO': 4,
  'RISCO MUITO ALTO': 5,
  'RISCO MÉDIO': 12,
  'CPF | Pendente de Regularização': 1,
  'RISCO BAIXO': 5}}

In [8]:
csv_columns = ['TIPO','RMB','RB', 'RM', 'RAM', 'RMA', 'RI', 'NA', 'CPF-SUS', 'CPF-N-CONF', 'CPF-NULO', 'SO', 'None']

with open('relatorio-risco.csv', 'w', newline='', encoding='utf-8') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    spamwriter.writerow(csv_columns)
    for key in relatorio:
        listaRow = list()
        listaRow.append(key)
        if 'RISCO MUITO BAIXO' in relatorio[key]:
            listaRow.append(relatorio[key]['RISCO MUITO BAIXO'])
        else: 
            listaRow.append('-')
        if 'RISCO BAIXO' in relatorio[key]:
            listaRow.append(relatorio[key]['RISCO BAIXO'])
        else: 
            listaRow.append('-')
        if 'RISCO MÉDIO' in relatorio[key]:
            listaRow.append(relatorio[key]['RISCO MÉDIO'])
        else: 
            listaRow.append('-')
        if 'RISCO ACIMA DA MÉDIA' in relatorio[key]:
            listaRow.append(relatorio[key]['RISCO ACIMA DA MÉDIA'])
        else: 
            listaRow.append('-')
        if 'RISCO MUITO ALTO' in relatorio[key]:
            listaRow.append(relatorio[key]['RISCO MUITO ALTO'])
        else: 
            listaRow.append('-')
        if 'RENDA INCOMPATÍVEL' in relatorio[key]:
            listaRow.append(relatorio[key]['RENDA INCOMPATÍVEL'])
        else: 
            listaRow.append('-')
        if 'NÃO APURADO' in relatorio[key]:
            listaRow.append(relatorio[key]['NÃO APURADO'])
        else: 
            listaRow.append('-')
        if 'CPF | Suspenso' in relatorio[key]:
            listaRow.append(relatorio[key]['CPF | Suspenso'])
        else: 
            listaRow.append('-')
        if 'CPF | Não confirmado no cadastro até esta data' in relatorio[key]:
            listaRow.append(relatorio[key]['CPF | Não confirmado no cadastro até esta data'])
        else: 
            listaRow.append('-')
        if 'CPF | nulo' in relatorio[key]:
            listaRow.append(relatorio[key]['CPF | nulo'])
        else: 
            listaRow.append('-')
        if 'SUSPEITA DE ÓBITO' in relatorio[key]:
            listaRow.append(relatorio[key]['SUSPEITA DE ÓBITO'])
        else: 
            listaRow.append('-')
        if None in relatorio[key]:
            listaRow.append(relatorio[key][None])
        else: 
            listaRow.append('-')
        spamwriter.writerow(listaRow)
        
print('Script finalizado com sucesso!')

Script finalizado com sucesso!
